Capítulo 6

6.2 Patrones de filtrado.

6.2.2 Filtro valores más altos

Ejemplo de patrón de filtrado para extraer los N valores más altos.  

In [2]:
%%writefile topN.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re

TopN=10 #Los N valores más altos que vamos a extraer

class topN(MRJob):
    
    #Método que extrae los N valores más altos de una lista de valores.
    def extraeTopN(self,listaValores):
        valoresMasAltos = []
        cuenta=0
        #Comprobamos que el número de valores Top que necesitamos no sea mayor que el tamaño de la lista
        if TopN>len(listaValores):#si lo es, N lo limitamos al tamaño máximo de la lista.
            nValores=len(listaValores)
        else:
            nValores=TopN
        
        #Buscamos los N valores distintos más altos. Empezamos por el final de la lista
        #que al estar ordenada, los valores más altos están al final. Pero puede haber valores repetidos
        #y eso es lo que vamos a hacer con estas líneas de código, evitar que se repitan valores.
        
        
        posicion=len(listaValores)-1 #Almacenamos la última posición de la lista
        
        while cuenta<nValores: #Como vamos a coger los N valores, "cuenta" la limitamos al número de valores 
                                #que quiera localizar
            try:
                if listaValores[posicion] not in valoresMasAltos: #Si el valor no está en los valores más altos, lo agregamos
                    valoresMasAltos.append(listaValores[posicion])
                    cuenta+=1 # si hemos agregado un valor aumentamos cuenta 
                    
                posicion-=1 # tanto si hemos agregado un valor como si no, disminuimos la posición 
                        # de la lista para buscar un nuevo valor. Puede darse el caso que posición llegue a -1
                        #(cuenta y posición no van al unísono, sobre todo cuando hay valores repetidos)
                        #esto genera una excepción de índice fuera de rango. Es recogida por el try-catch y devuelve
                        #los valores correctos por que ha terminado.
            except:
                break
                
        return valoresMasAltos
    
    #Inicializamos la estructura que nos permitirá contener los valores que nos llegan del Map
    def mapper_init(self):
        self.topNmap = []
    
    def mapper(self,_, line):
        self.registro=line.rstrip('/n').split(';') 
        encontrado=re.search('[a-zA-Z]',self.registro[5])#En este caso el valor lo tengo en la posción 5 de la línea
        #Preguntamos si ha encontrado la expresion regular. Si no la ha encontrado, procesamos dato
        if encontrado==None:
            self.topNmap.append(float(self.registro[5]))
        
    def mapper_final(self):
        self.topNmap.sort() #se ordena la lista
        #La función extraeTopN extrae los 10 elementos últimos de la lista, que se envían a reducer
        yield "TopNmap:", self.extraeTopN(self.topNmap)
        
 #===============================REDUCER=======================================     
        
    def reducer(self, key, values):
        listaValores=[]
        
        # metemos los valores en una lista
        for valores in values:
            listaValores.extend(valores)
        
        # ordenamos la lista    
        listaValores.sort()
        yield "TopNreducer:", self.extraeTopN(listaValores)
        
if __name__ == '__main__':
    topN.run()

Writing topN.py


In [3]:
!python topN.py archivos_datos/ventas_UK.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/topN.root.20201214.191225.462453
Running step 1 of 1...
job output is in /tmp/topN.root.20201214.191225.462453/output
Streaming final output from /tmp/topN.root.20201214.191225.462453/output...
"TopNreducer:"	[38970.0, 17836.46, 16888.02, 16453.71, 13541.33, 13474.79, 11586.5, 11062.06, 8286.22, 8142.75]
Removing temp directory /tmp/topN.root.20201214.191225.462453...


EJECUCIÓN EN EL CLUSTER:

In [4]:
! python topN.py hdfs:///archivos_datos/ventas_UK.csv -r hadoop --python-bin /opt/anaconda/bin/python

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/topN.root.20201214.191338.134298
uploading working dir files to hdfs:///user/root/tmp/mrjob/topN.root.20201214.191338.134298/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/topN.root.20201214.191338.134298/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7243219805135454558.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Total input paths to process : 1
  numb